# 第十二章 量化交易机器学习模型

## 实战一：用lightGBM 构建量化交易预测模型

本实战将利用lightGBM这种集成模型构建一个基于价格预测的量化交易模型，在本小节中您将主要学会以及巩固以下内容：

* 数据导入
* 模型构建
* 因子计算
* 可视化输出

在实战中我们会主要使用到以下Python第三方库或是课程编写的YYIT量化库

* [numpy](https://numpy.org/doc/)           矩阵运算库
* [pandas](https://pandas.pydata.org/docs/) 数据分析库
* [lightgbm](https://lightgbm.readthedocs.io/en/latest/) 集成模型计算库
* [matplotlib](https://matplotlib.org/contents.html) 绘图库
* [plotly](https://plot.ly/~plotlu) 交互式绘图库

## 第一节 数据导入

在这一节中我们将使用pandas以及get_data.py从csv文件中获取已有的量化交易数据。

提示：我们课程老师在get_data.py中封装了load_kline_data()函数。该函数具有两个参数：第一个参数SYMBOL表示的想导入的标的物名称。第二个参数PERIOD表示了想导入的对应k线周期(1min, 5min, 10min, 1H, 等等)。

任务1: 从数据文件中加载一个你想要导入的标的

In [2]:
kline_df = kline_df[kline_df['volume'] != 0]

# TODO1：导入数据

kline_df = load_kline_data(SYMBOL, PERIOD)
kline_df = kline_df[kline_df['volume'] != 0]

kline_df.tail()

# 第二节 添加因子
## 2.1 导入因子

在这一节中，我们将使用老师为大家封装的Factors类，用以方便的导入量化交易因子。

首先简单的介绍一下Factors类，这个类位于factors_repo文件夹内，用以计算可由k线计算出的量化交易因子。
下面给大家一个简单的例子

``` python
# 引入Factors类
from factors_repo import Factors

# 初始化Factors类
factors_manager = Factors()

# 添加周期为 14的RSI因子
factors_manager.add_factor('barRSI_14', timeperiod=14)

# 计算已经添加的因子
factors_df = factors_manager.cal_candle_factors(kline_df)
``` 

任务2：添加 至少一种均线因子，一种波动率因子，一种震荡因子，以及一种其他因子


In [ ]:
# TODO2: 
from factors_repo import Factors

factors_manager = Factors()
factors_manager.add_factor('barRSI_14', timeperiod=14)
factors_df = factors_manager.cal_candle_factors(kline_df)


## 2.2 异常值处理

在这里你需要对你的因子进行观察，以确定是否每一种因子都没有下列情况：
* 空值（NA）
* 无穷大/无穷小 （inf/-inf）
* 选做 偏离值 （与正常分布明显不符）

任务3: 对上述的因子进行清洗, 确保没有空值与无穷大无穷小的值。
如果有兴趣的同学可以继续对每一列的值进行进一步探究，判断这些值是否有除了空值与无穷大无穷小之外的异常值

In [ ]:
# TODO3:
factors_df = factors_df.dropna(0).replace(np.inf,0).replace(-np.inf,0)


# 第三节 数据集构建

在这一节，我们将进行构建我们机器学习中使用到的数据集。我们知道在机器学习中（尤其是监督学习）,要构建一个模型可用的数据集,我们要给数据集喂入模型的输入（在这个实战中其实就是我们的因子）与模型的标签（未来收益，未来波动率等等）

那么在之前的因子计算环节我们其实已经将我们数据集的输入X完成了，那么接下来我们就需要数据集的标签y。

## 3.1 标签的计算

在量化交易的机器学习中，有很多各种各样的标签计算方式，包括但不限于（未来收益（回归），未来波动（回归），事件型（分类）等等）。其中最直观也是最简单的一种标签计算方式便是计算未来的收益。

任务4: 利用python完成一个计算未来收益的函数, 该函数的输入为我们的k线
提示: 可以利用当前点t的收盘价-前一点t-1的收盘价计算出当前点的收益，那么对于t-1时间点来说的未来收益就是close(t)-close(t-1)
 

In [4]:
def profit_label(bars):
    """计算label profit"""
    # TODO3:

    close = bars['close'].values
    x = close
    x_roll = np.roll(close, 1)
    x = x - x_roll

    return x 

## 3.1 (选做) 更多的标签
上面我们计算的profit_label是基于未来收益的回归标签，但是很多时候我们可以使用分类标签来定义市场是一个可买入，持有，和可卖出状态的标签。即我们可以将市场分为三类-1（可卖出），0（可持有/或观测），1（可买入）。

我们之前利用了2倍方差构建了bolling通道，同样我们在构建分类因子的时候也可以利用相同的思想。
当未来收益超过 n*方差时 我们将这样的状态标记为1。当未来收益在-n*方差和n*方差之间时我们将其标记为0。当未来收益小于-n*方差时我们将其标记为 -1。

In [ ]:
def profit_class_label(bars):
    # TODO3.1:

    return x

# 3.1 计算label
任务5: 将label利用数据计算出来

In [ ]:
#TODO4:



## 3.2 数据集的分割

在量化交易机器学习中，我们往往会将一段时间内的数据有序的进行训练集和测试集的分割。
在机器学习库Scikit-learn中，提供了很好的机器学习数据集分割工具。我们可以用sklearn（Scikit-learn的更通俗称法）中的train_test_split函数进行数据集的分割。

例如,这里我们将数据集有序的分割为了2份，比例是训练集80%/测试集20%

``` python
# 导入 train_test_split
from sklearn.model_selection import train_test_split

# 进行数据集分割
X_train, X_test, y_train, y_test = train_test_split(data_array, label, test_size=0.2)

``` 
在过去的传统机器学习中，由于样本之间的分布一般较为趋同（测试集和训练集样本遵循或近似独立同分布）所以一般我们只进行train_test分割即可。
但是由于金融时间序列本身的不可预测性与多变性导致不同时间段的数据分布并不遵循独立同分布状态，所以我们需要额外的样本外数据以防止模型的过拟合（针对测试集的过拟合），其中最简单的办法是我们在其中额外再跳出一部分数据作为样本外预测。

任务6: 我们需要将数据集分割成80%-10%-10%的三份。


In [ ]:
# TODO:5

# 第四节 模型训练
这一节我们将学习使用如何利用lightGBM进行集成机器学习模型的训练和预测。由于lightGBM具有两套api标准，第一套为与sklearn高度相似与兼容的sklearn API,另外一套则是lightGBM的原生api。在课程里为了学习的高效性，我们使用了sklearn的api进行教学。

## 4.1 模型生成

相比于lightGBM的原生的api，sklearn的兼容api使用较为简单。我们可以使用lgb.LGBMRegressor()创建一个回归模型。在模型建立的时候有一些参数是我们比较注重也比较例如叶节点数量，学习率，迭代次数等等

例如

``` python
#导入 lightGBM Regressor
from lightGBM import LGBMRegressor

lgboost = lgb.LGBMRegressor(num_leaves=10,
                            learning_rate=0.05,
                            n_estimators=200)

``` 

任务7: 建立自己的机器学习模型, 在任务中可以修改学习率，迭代次数等。
ps: 一个有效的机器学习模型的迭代次数往往由几千次到上万次不等。

In [ ]:
# TODO6:



## 4.2 模型训练
接下来我们进行模型的训练，我们将利用lgb的sklearn中fit API将数据导入模型中。

任务8: 将之前的数据填入我们的机器学习模型中


In [ ]:
# TODO7:

## 4.2 模型训练（选做）

上面我们简单的进行了模型的训练，但是上述的简单训练并不十分灵活。往往在机器学习的训练中我们需要自己定义损失函数，early_stop等等一些灵活的配置措施以提升我们的训练效率和训练结果。

任务9: 定义rmspe损失函数


In [2]:
# TODO8:
def rmspe(y, yhat):

    return 
def rmspe_lgb(yhat, y):

    return 

SyntaxError: EOL while scanning string literal (<ipython-input-2-7f2281bb766d>, line 2)

任务10: 在这一部分我们需要对模型的损失函数，训练可视化步骤，自动停止等进行定义。

In [ ]:
# TODO9:

## 4.3 模型预测以及模型预测可视化

### 4.3.1 模型预测

我们可以利用predict计算出我们的预测结果

任务11: 利用predicth函数进行模型的预测，将我们前面的验证集的X输入进模型中求得预测的y_hat


In [ ]:
# TODO10:



## 4.3.2 因子重要性可视化

量化因子的可视化也是用来筛选因子的一个重要方法，lightGBM提供了方便的因子重要性可视化工具用以判断因子是否有效以及因子归因。

任务12：利用_feature_importances绘制出因子可视化图片


In [ ]:
# TODO11: